In [17]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_sam
from functional_map import compute_surface_map

In [18]:
device = torch.device('cuda:0')
torch.cuda.set_device(device)
num_views = 100
H = 512
W = 512
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [19]:
def compute_features(device, pipe, dino_model, m, prompt):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        pipe=pipe, 
        dino_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        num_images_per_prompt=num_images_per_prompt,
        use_normal_map=use_normal_map,
        use_sam=True,
        use_diffusion=False,
    )
    return features.cpu()

In [20]:
pipe = init_pipe(device)
dino_model = init_sam(device)

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [21]:
source_file_path = "meshes/cow.obj"
target_file_path = "meshes/camel.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [22]:
f_source = compute_features(device, pipe, dino_model, source_mesh, "cow")

Rendering complete


  0%|                                                                                                           | 0/100 [00:04<?, ?it/s]


AttributeError: 'SAM2Base' object has no attribute 'get_intermediate_layers'

In [ ]:
f_target = compute_features(device, pipe, dino_model, target_mesh, "camel")

In [ ]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]

In [ ]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Apply functional map on features

In [ ]:
source_file_path = "meshes/cat.off"
target_file_path = "meshes/lion.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "cat")
f_target = compute_features(device, pipe, dino_model, target_mesh, "lion")

In [ ]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [ ]:
surface_map = compute_surface_map(source_file_path, target_file_path, f_source.numpy(), f_target.numpy())

In [ ]:
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[surface_map.cpu().numpy()]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Part segmentation

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner
segments2 = kmeans.predict(f_target)

In [ ]:
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [ ]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [17]:
source_file_path = "meshes/posed_human.off"
target_file_path = "meshes/cat.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "naked human")
f_target = compute_features(device, pipe, dino_model, target_mesh, "cat")

Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:24<00:00,  2.04s/it]


Number of missing features:  3
Copied features from nearest vertices
Time taken in mins:  3.526184153556824
Rendering complete


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:25<00:00,  2.06s/it]

Number of missing features:  140
Copied features from nearest vertices
Time taken in mins:  3.5948370456695558


In [18]:
k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner

segments2 = kmeans.predict(f_target)
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [19]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  